In [2]:
import pandas as pd
import numpy as np

In [3]:
def extract_data():
    """
    Extrae datos de múltiples archivos CSV
    
    Returns:
        dict: Diccionario con año como clave y DataFrame como valor
    """
    print("=" * 60)
    print("FASE 1: EXTRACCIÓN DE DATOS")
    print("=" * 60)
    
    files = {
        2015: '../data/2015.csv',
        2016: '../data/2016.csv',
        2017: '../data/2017.csv',
        2018: '../data/2018.csv',
        2019: '../data/2019.csv'
    }
    
    raw_data = {}
    
    for year, file in files.items():
        try:
            df = pd.read_csv(file)
            raw_data[year] = df
            print(f"✓ {year}: {len(df)} registros, {len(df.columns)} columnas")
        except Exception as e:
            print(f"✗ Error al cargar {file}: {str(e)}")
    
    print(f"\n✓ Extracción completada: {len(raw_data)} archivos cargados\n")
    return raw_data

In [15]:
def transform_data(raw_data):
    """
    Transforma y estandariza los datos de todos los años
    
    Args:
        raw_data (dict): Diccionario con DataFrames por año
        
    Returns:
        pd.DataFrame: DataFrame combinado y transformado
    """
    print("=" * 60)
    print("FASE 2: TRANSFORMACIÓN DE DATOS")
    print("=" * 60)
    
    # Mapeo unificado de columnas
    column_map = {
        # Identificadores
        'Country': 'Country',
        'Country or region': 'Country',
        'Region': 'Region',
        
        # Score
        'Happiness Rank': 'Happiness_Rank',
        'Happiness.Rank': 'Happiness_Rank',
        'Overall rank': 'Happiness_Rank',
        'Happiness Score': 'Happiness_Score',
        'Happiness.Score': 'Happiness_Score',
        'Score': 'Happiness_Score',
        
        # GDP
        'Economy (GDP per Capita)': 'GDP_per_Capita',
        'Economy..GDP.per.Capita.': 'GDP_per_Capita',
        'GDP per capita': 'GDP_per_Capita',
        
        # Social Support
        'Family': 'Social_Support',
        'Social support': 'Social_Support',
        
        # Life Expectancy
        'Health (Life Expectancy)': 'Healthy_Life_Expectancy',
        'Health..Life.Expectancy.': 'Healthy_Life_Expectancy',
        'Healthy life expectancy': 'Healthy_Life_Expectancy',
        
        # Freedom
        'Freedom': 'Freedom',
        'Freedom to make life choices': 'Freedom',
        
        # Corruption
        'Trust (Government Corruption)': 'Perceptions_of_Corruption',
        'Trust..Government.Corruption.': 'Perceptions_of_Corruption',
        'Perceptions of corruption': 'Perceptions_of_Corruption',
        
        # Generosity
        'Generosity': 'Generosity',
    }
    
    # Mapeo de nombres de países
    country_map = {
        'Taiwan Province of China': 'Taiwan',
        'Hong Kong S.A.R., China': 'Hong Kong',
        'Trinidad & Tobago': 'Trinidad and Tobago',
        'North Cyprus': 'Northern Cyprus',
        'Northern Cyprus': 'Northern Cyprus',
        'Somaliland Region': 'Somaliland region',
        'Palestinian Territories': 'Palestine',
        'Congo (Kinshasa)': 'Democratic Republic of Congo',
        'Congo (Brazzaville)': 'Republic of Congo',
        'Ivory Coast': "Cote d'Ivoire",
        'Macedonia': 'North Macedonia',
        'North Macedonia': 'North Macedonia',
        'Swaziland': 'Eswatini'
    }
    
    # Crear mapeo de regiones desde 2015 y 2016
    region_map = {}
    for year in [2015, 2016]:
        if year in raw_data:
            df_temp = raw_data[year].rename(columns=column_map)
            if 'Country' in df_temp.columns and 'Region' in df_temp.columns:
                df_temp['Country'] = df_temp['Country'].str.strip()
                df_temp['Country'] = df_temp['Country'].replace(country_map)
                region_dict = df_temp.set_index('Country')['Region'].to_dict()
                region_map.update(region_dict)
    
    print(f"✓ Mapeo de regiones creado: {len(region_map)} países\n")
    
    # Transformar cada año
    transformed_dfs = []
    
    for year, df in raw_data.items():
        print(f"--- Procesando {year} ---")
        
        # 1. Renombrar columnas
        df_clean = df.rename(columns=column_map)
        print(f"  ✓ Columnas renombradas")
        
        # 2. Estandarizar nombres de países
        if 'Country' in df_clean.columns:
            df_clean['Country'] = df_clean['Country'].str.strip()
            df_clean['Country'] = df_clean['Country'].replace(country_map)
            print(f"  ✓ Nombres de países estandarizados")
        
        # 3. Asignar regiones faltantes
        if 'Region' not in df_clean.columns:
            df_clean['Region'] = df_clean['Country'].map(region_map)
            print(f"  ✓ Regiones asignadas desde años 2015-2016")
        elif df_clean['Region'].isnull().any():
            # Rellenar regiones faltantes
            df_clean['Region'].fillna(df_clean['Country'].map(region_map), inplace=True)
            print(f"  ✓ Regiones faltantes rellenadas")
        else:
            print(f"  ✓ Regiones ya presentes")
        
        # 4. Añadir columna Year
        df_clean['Year'] = year
        print(f"  ✓ Columna Year añadida")
        
        # 5. Seleccionar solo las columnas que necesitamos
        columns_to_keep = [
            'Country', 'Region', 'Year', 'Happiness_Score',
            'GDP_per_Capita', 'Social_Support', 'Healthy_Life_Expectancy',
            'Freedom', 'Generosity', 'Perceptions_of_Corruption'
        ]
        
        # Mantener solo las columnas que existen
        available_columns = [col for col in columns_to_keep if col in df_clean.columns]
        df_clean = df_clean[available_columns]
        
        print(f"  ✓ Columnas seleccionadas: {len(available_columns)}")
        print(f"  ✓ Registros procesados: {len(df_clean)}\n")
        
        transformed_dfs.append(df_clean)
    
    # Combinar todos los años
    print("--- Combinando todos los años ---")
    final_df = pd.concat(transformed_dfs, ignore_index=True)
    
    # Ordenar por año y país
    final_df = final_df.sort_values(['Year', 'Country']).reset_index(drop=True)
    
    print(f"✓ Datos combinados exitosamente")
    print(f"✓ Total de registros: {len(final_df)}")
    print(f"✓ Países únicos: {final_df['Country'].nunique()}")
    print(f"✓ Años: {sorted(final_df['Year'].unique())}")
    print(f"✓ Columnas finales: {list(final_df.columns)}\n")

    # Manejo de valores nulos después del merge
    print("--- Revisando y manejando valores nulos ---")
    
    # Contar nulos por columna
    null_counts = final_df.isnull().sum()
    total_nulls = null_counts.sum()
    
    if total_nulls > 0:
        print(f"⚠ Se encontraron {total_nulls} valores nulos:")
        for col, count in null_counts[null_counts > 0].items():
            percentage = (count / len(final_df)) * 100
            print(f"  - {col}: {count} ({percentage:.2f}%)")
        
        print("\n--- Aplicando estrategias de imputación ---")
        
        # Estrategia 1: Regiones faltantes
        if final_df['Region'].isnull().any():
            nulls_before = final_df['Region'].isnull().sum()
            # Intentar rellenar con la región más común del país en otros años
            for country in final_df[final_df['Region'].isnull()]['Country'].unique():
                common_region = final_df[final_df['Country'] == country]['Region'].mode()
                if len(common_region) > 0:
                    final_df.loc[(final_df['Country'] == country) & (final_df['Region'].isnull()), 'Region'] = common_region[0]
            
            # Si aún quedan nulos, asignar 'Unknown'
            final_df['Region'].fillna('Unknown', inplace=True)
            nulls_after = final_df['Region'].isnull().sum()
            print(f"  ✓ Region: {nulls_before} → {nulls_after} nulos")
        
        # Estrategia 2: Valores numéricos - imputar con mediana del país
        numeric_cols = ['Happiness_Score', 'GDP_per_Capita', 'Social_Support', 
                       'Healthy_Life_Expectancy', 'Freedom', 'Generosity', 
                       'Perceptions_of_Corruption']
        
        for col in numeric_cols:
            if col in final_df.columns and final_df[col].isnull().any():
                nulls_before = final_df[col].isnull().sum()
                
                # Intentar imputar con la mediana del mismo país en otros años
                for country in final_df[final_df[col].isnull()]['Country'].unique():
                    country_median = final_df[final_df['Country'] == country][col].median()
                    if pd.notna(country_median):
                        final_df.loc[(final_df['Country'] == country) & (final_df[col].isnull()), col] = country_median
                
                # Si aún quedan nulos, usar mediana de la región
                for region in final_df[final_df[col].isnull()]['Region'].unique():
                    if pd.notna(region) and region != 'Unknown':
                        region_median = final_df[final_df['Region'] == region][col].median()
                        if pd.notna(region_median):
                            final_df.loc[(final_df['Region'] == region) & (final_df[col].isnull()), col] = region_median
        final_nulls = final_df.isnull().sum().sum()
        if final_nulls == 0:
            print(f"\n✓ Todos los valores nulos han sido manejados exitosamente")
        else:
            print(f"\n⚠ Quedan {final_nulls} valores nulos sin resolver")
            print(final_df.isnull().sum()[final_df.isnull().sum() > 0])
    else:
        print(f"✓ No se encontraron valores nulos en el dataset combinado")
    
    print()
    return final_df


In [16]:
def load_data(df, output_file='data/happiness_merged_2015-2019.csv'):
    """
    Carga los datos transformados en un archivo CSV
    
    Args:
        df (pd.DataFrame): DataFrame con los datos transformados
        output_file (str): Nombre del archivo de salida
        
    Returns:
        None
    """
    print("=" * 60)
    print("FASE 3: CARGA DE DATOS")
    print("=" * 60)
    
    try:
        df.to_csv(output_file, index=False, encoding='utf-8')
        print(f"✓ Datos guardados exitosamente en: {output_file}")
        print(f"✓ Tamaño del archivo: {len(df)} filas x {len(df.columns)} columnas")
        
    except Exception as e:
        print(f"✗ Error al guardar el archivo: {str(e)}")
    
    print("\n" + "=" * 60)
    print("✓ PROCESO ETL COMPLETADO")
    print("=" * 60)

In [17]:
# ==================== EJECUCIÓN ====================
if __name__ == "__main__":
    # Paso 1: Extraer datos
    raw_data = extract_data()
    
    # Paso 2: Transformar datos
    transformed_df = transform_data(raw_data)
    
    # Paso 3: Cargar datos
    load_data(transformed_df, output_file='data/happiness_merged_2015-2019.csv')
    
    print("\n✓ Archivo listo para crear el modelo!")

FASE 1: EXTRACCIÓN DE DATOS
✓ 2015: 158 registros, 12 columnas
✓ 2016: 157 registros, 13 columnas
✓ 2017: 155 registros, 12 columnas
✓ 2018: 156 registros, 9 columnas
✓ 2019: 156 registros, 9 columnas

✓ Extracción completada: 5 archivos cargados

FASE 2: TRANSFORMACIÓN DE DATOS
✓ Mapeo de regiones creado: 163 países

--- Procesando 2015 ---
  ✓ Columnas renombradas
  ✓ Nombres de países estandarizados
  ✓ Regiones ya presentes
  ✓ Columna Year añadida
  ✓ Columnas seleccionadas: 10
  ✓ Registros procesados: 158

--- Procesando 2016 ---
  ✓ Columnas renombradas
  ✓ Nombres de países estandarizados
  ✓ Regiones ya presentes
  ✓ Columna Year añadida
  ✓ Columnas seleccionadas: 10
  ✓ Registros procesados: 157

--- Procesando 2017 ---
  ✓ Columnas renombradas
  ✓ Nombres de países estandarizados
  ✓ Regiones asignadas desde años 2015-2016
  ✓ Columna Year añadida
  ✓ Columnas seleccionadas: 10
  ✓ Registros procesados: 155

--- Procesando 2018 ---
  ✓ Columnas renombradas
  ✓ Nombres de p

C:\Users\annie\AppData\Local\Temp\ipykernel_6912\2695442513.py:172: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['Region'].fillna('Unknown', inplace=True)


In [18]:
df_merged = pd.read_csv('data/happiness_merged_2015-2019.csv')

In [19]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Country                    782 non-null    object 
 1   Region                     782 non-null    object 
 2   Year                       782 non-null    int64  
 3   Happiness_Score            782 non-null    float64
 4   GDP_per_Capita             782 non-null    float64
 5   Social_Support             782 non-null    float64
 6   Healthy_Life_Expectancy    782 non-null    float64
 7   Freedom                    782 non-null    float64
 8   Generosity                 782 non-null    float64
 9   Perceptions_of_Corruption  782 non-null    float64
dtypes: float64(7), int64(1), object(2)
memory usage: 61.2+ KB
